# Chapter 13: Practical Text Processing

**Chapter 13 - Learning Python**

This notebook bridges the gap between regex theory and real-world usage.
We cover email validation, compiled patterns, data extraction from structured
text, efficient iteration with `finditer`, when to prefer string methods over
regex, and common pitfalls to avoid.

## Key Concepts
- **Email validation**: building a practical (not RFC-complete) email pattern
- **`re.split`**: splitting strings on complex delimiters
- **`re.compile`**: pre-compiling patterns for reuse and readability
- **Data extraction**: parsing logs, CSVs, and structured text
- **`re.finditer`**: memory-efficient iteration over matches
- **String methods vs regex**: choosing the right tool
- **Common pitfalls**: catastrophic backtracking and over-engineering

## Email Validation

Email validation with regex is a classic exercise. A fully RFC 5322-compliant
pattern is extraordinarily complex. In practice, a reasonable approximation
covers the vast majority of valid addresses.

In [ ]:
import re

# A practical email validation pattern
# Matches: local part @ domain . tld
# Local part: letters, digits, dots, underscores, hyphens, plus
# Domain: letters, digits, hyphens, with at least one dot
# TLD: 2 or more letters
EMAIL_PATTERN = re.compile(
    r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
)


def is_valid_email(email: str) -> bool:
    """Check if a string looks like a valid email address."""
    return bool(EMAIL_PATTERN.match(email))


test_emails = [
    "user@example.com",         # valid
    "alice.bob+tag@work.org",   # valid (dots, plus in local)
    "name@sub.domain.co.uk",    # valid (subdomain)
    "@missing-local.com",       # invalid (no local part)
    "no-at-sign.com",           # invalid (no @)
    "user@.com",                # invalid (dot after @)
    "user@domain",              # invalid (no TLD)
    "user@domain.a",            # invalid (TLD too short)
]

print("Email validation results:")
for email in test_emails:
    status = "valid" if is_valid_email(email) else "INVALID"
    print(f"  {email:35s} -> {status}")

print("\nNote: For production use, consider the 'email-validator' package.")
print("Regex can never fully validate emails per RFC 5322.")

## `re.split` -- Splitting on Complex Patterns

`str.split()` works on literal delimiters. `re.split()` lets you split on
patterns, handling multiple or variable delimiters in one call.

In [ ]:
import re

# Problem: split on commas, semicolons, or pipes (with optional whitespace)
data = "apple, banana; cherry | date,  elderberry;fig"

# str.split can only handle one delimiter
print(f"str.split(','):  {data.split(',')}")

# re.split handles multiple delimiters at once
items = re.split(r"[,;|]\s*", data)
print(f"re.split:        {items}")

# Split on one or more whitespace characters (like str.split() but explicit)
messy = "  hello   world\tfoo\n\nbar  "
clean = re.split(r"\s+", messy.strip())
print(f"\nWhitespace split: {clean}")

# Capturing groups in the split pattern KEEP the delimiters
equation = "3+4-5*2/1"
parts = re.split(r"([+\-*/])", equation)
print(f"\nSplit with captured delimiters: {parts}")

# maxsplit parameter: limit the number of splits
log_line = "ERROR:2024-01-15:disk:full:critical"
level, date, rest = re.split(r":", log_line, maxsplit=2)
print(f"\nmaxsplit=2: level={level!r}, date={date!r}, rest={rest!r}")

## Compiled Patterns with `re.compile`

`re.compile(pattern)` returns a reusable `Pattern` object. Benefits:
- **Performance**: the pattern is compiled once and reused across many calls
- **Readability**: the pattern gets a descriptive variable name
- **Organization**: group related flags and patterns together

The compiled object has the same methods: `.match()`, `.search()`, `.findall()`,
`.sub()`, `.split()`, etc.

In [ ]:
import re

# Compile a pattern once, use it many times
WORD_PATTERN = re.compile(r"\b[A-Z][a-z]+\b")
IP_PATTERN = re.compile(
    r"\b(?:\d{1,3}\.){3}\d{1,3}\b"
)

# Use the compiled pattern's methods directly
texts = [
    "Alice met Bob in Paris",
    "the quick brown fox jumped",
    "Charlie went to New York and London",
]

print("Capitalized words (compiled pattern):")
for text in texts:
    matches = WORD_PATTERN.findall(text)
    print(f"  {text!r} -> {matches}")

# IP address matching
log = """Connection from 192.168.1.1 accepted
Failed login from 10.0.0.5
Timeout on 172.16.0.100"""

print(f"\nIP addresses found: {IP_PATTERN.findall(log)}")

# Compiled patterns with flags
COMMENT_PATTERN = re.compile(
    r"#.*$",          # '#' followed by anything to end of line
    re.MULTILINE,     # $ matches at each line end
)

code = """x = 42  # the answer
y = x + 1  # increment
print(y)  # output"""

stripped = COMMENT_PATTERN.sub("", code)
print(f"\nCode without comments:")
print(stripped)

## Extracting Data from Structured Text

Regex excels at parsing semi-structured text like log files, configuration
files, and simple data formats.

In [ ]:
import re
from typing import NamedTuple


class LogEntry(NamedTuple):
    timestamp: str
    level: str
    module: str
    message: str


LOG_PATTERN = re.compile(
    r"\[(?P<timestamp>[\d-]+ [\d:]+)\]\s+"
    r"(?P<level>DEBUG|INFO|WARNING|ERROR|CRITICAL)\s+"
    r"(?P<module>[\w.]+):\s+"
    r"(?P<message>.+)"
)


def parse_log(log_text: str) -> list[LogEntry]:
    """Parse structured log lines into LogEntry objects."""
    entries: list[LogEntry] = []
    for m in LOG_PATTERN.finditer(log_text):
        entries.append(LogEntry(**m.groupdict()))
    return entries


sample_log = """[2024-01-15 10:23:45] INFO server.main: Server started on port 8080
[2024-01-15 10:23:46] DEBUG server.auth: Loading auth configuration
[2024-01-15 10:24:01] WARNING server.db: Connection pool 80% full
[2024-01-15 10:24:15] ERROR server.handler: Request timeout after 30s
[2024-01-15 10:24:16] INFO server.handler: Retrying request"""

entries = parse_log(sample_log)
print(f"Parsed {len(entries)} log entries:\n")
for entry in entries:
    print(f"  [{entry.level:8s}] {entry.timestamp} | {entry.module}: {entry.message}")

# Filter: show only warnings and errors
issues = [e for e in entries if e.level in ("WARNING", "ERROR", "CRITICAL")]
print(f"\nIssues found: {len(issues)}")
for e in issues:
    print(f"  {e.level}: {e.message}")

## `re.finditer` -- Memory-Efficient Iteration

`re.findall` builds a list of all matches in memory. `re.finditer` returns
an **iterator** of `Match` objects, yielding one match at a time. This is
more memory-efficient for large texts and gives access to the full `Match`
object (position, groups, etc.).

In [ ]:
import re

text = """Scores: Alice=95, Bob=87, Charlie=92, Diana=88, Eve=96"""

# findall returns strings (or tuples) -- loses position information
names_scores = re.findall(r"(\w+)=(\d+)", text)
print(f"findall: {names_scores}")

# finditer returns Match objects -- preserves full match information
print("\nfinditer (with positions):")
for m in re.finditer(r"(?P<name>\w+)=(?P<score>\d+)", text):
    print(f"  {m.group('name'):10s} score={m.group('score'):>3s}  "
          f"at position {m.start()}-{m.end()}")

# finditer is an iterator -- it doesn't build the full list in memory
# This matters when processing large files
large_text = "word " * 10000  # simulate a large document
pattern = re.compile(r"\bword\b")

# Count matches without building a list
count = sum(1 for _ in pattern.finditer(large_text))
print(f"\nMatches in large text (via finditer): {count}")

# Compare: findall creates a list of 10000 strings in memory
# finditer yields one match at a time

## String Methods vs Regex: When to Use Which

Regex is powerful but not always the best tool. Python's built-in string
methods are faster and more readable for simple operations.

**Use string methods when:**
- Searching for a literal substring
- Splitting on a fixed delimiter
- Simple prefix/suffix checks
- Case conversion or stripping whitespace

**Use regex when:**
- The pattern is variable or complex
- You need capturing groups
- Multiple delimiters or flexible whitespace
- Validation against a structured format

In [ ]:
import re
import timeit

text = "Hello, World! Hello, Python!"

# --- Task 1: Check if string contains a substring ---
# String method (preferred -- simpler, faster)
has_python_str = "Python" in text
# Regex (overkill for this)
has_python_re = bool(re.search(r"Python", text))
print(f"Contains 'Python' (str): {has_python_str}")
print(f"Contains 'Python' (re):  {has_python_re}")

# --- Task 2: Check prefix ---
# String method (preferred)
starts_hello_str = text.startswith("Hello")
# Regex (unnecessary complexity)
starts_hello_re = bool(re.match(r"Hello", text))
print(f"\nStarts with 'Hello' (str): {starts_hello_str}")
print(f"Starts with 'Hello' (re):  {starts_hello_re}")

# --- Task 3: Replace fixed string ---
# String method (preferred)
replaced_str = text.replace("Hello", "Hi")
# Regex
replaced_re = re.sub(r"Hello", "Hi", text)
print(f"\nReplace (str): {replaced_str}")
print(f"Replace (re):  {replaced_re}")

# --- Task 4: Complex pattern -- regex is appropriate ---
# Extract version numbers like 'v1.2.3' or 'v10.0.1'
changelog = "Released v1.2.3, hotfix v1.2.4, major v2.0.0"
# No clean way to do this with string methods!
versions = re.findall(r"v\d+\.\d+\.\d+", changelog)
print(f"\nVersions (regex is the right tool): {versions}")

# --- Performance comparison ---
sample = "abc" * 1000 + "target" + "xyz" * 1000
t_str = timeit.timeit(lambda: "target" in sample, number=10000)
t_re = timeit.timeit(lambda: re.search(r"target", sample), number=10000)
print(f"\nPerformance (10k iterations):")
print(f"  'in' operator: {t_str:.4f}s")
print(f"  re.search:     {t_re:.4f}s")
print(f"  String method is ~{t_re / t_str:.1f}x faster for literal search")

## Common Pitfalls

### 1. Catastrophic Backtracking
Certain patterns with nested quantifiers can cause the regex engine to
explore an exponential number of paths, effectively hanging your program.

### 2. Over-Engineering with Regex
Not every text problem needs regex. Using regex for tasks that string
methods handle cleanly makes code harder to read and maintain.

### 3. Forgetting Raw Strings
Without `r"..."`, backslash sequences are interpreted by Python before
reaching the regex engine.

In [ ]:
import re
import time

# --- Pitfall 1: Catastrophic backtracking ---
# BAD pattern: nested quantifiers on overlapping character classes
# Pattern: (a+)+ matching against 'aaaaaaaaaaab'
# The engine tries exponentially many ways to divide 'a's among groups

print("Catastrophic backtracking demonstration:")
print("  BAD pattern: r'(a+)+b' with input 'aaa...ab' (no match case)")

# Safe: short input
safe_input = "a" * 10 + "b"
start = time.time()
re.match(r"(a+)+b", safe_input)
print(f"  10 a's + 'b' (match):   {time.time() - start:.6f}s")

# Dangerous: input that does NOT match causes backtracking
danger_input = "a" * 20 + "c"  # no 'b', so engine backtracks
start = time.time()
re.match(r"(a+)+b", danger_input)
elapsed = time.time() - start
print(f"  20 a's + 'c' (no match): {elapsed:.6f}s")

# The fix: use atomic patterns or possessive quantifiers
# In Python, rewrite to avoid nested quantifiers
start = time.time()
re.match(r"a+b", danger_input)  # Fixed: no nested quantifier
print(f"  Fixed pattern (a+b):     {time.time() - start:.6f}s")

print("\n--- Pitfall 2: Over-engineering ---")
# DON'T use regex for simple checks
filename = "report.pdf"

# Overcomplicated regex
is_pdf_regex = bool(re.search(r"\.pdf$", filename))
# Clean string method
is_pdf_simple = filename.endswith(".pdf")
print(f"  regex way:  {is_pdf_regex}")
print(f"  simple way: {is_pdf_simple}  (prefer this!)")

print("\n--- Pitfall 3: Forgetting raw strings ---")
# \b in a normal string is a backspace character
bad_pattern = "\bword\b"       # \b = backspace!
good_pattern = r"\bword\b"     # \b = word boundary
text = "a word here"
print(f"  Without r'': {re.search(bad_pattern, text)}")
print(f"  With r'':    {re.search(good_pattern, text).group()}")

## Summary

### Practical Patterns
- **Email validation**: `r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"`
- For production, use dedicated validation libraries rather than regex alone

### `re.split` and `re.compile`
- `re.split(pattern, string)` handles multiple/variable delimiters
- `re.compile(pattern)` creates a reusable `Pattern` object for performance and readability

### Data Extraction
- Named groups + `finditer` + `NamedTuple` is a clean pattern for parsing structured text
- `re.finditer` is memory-efficient: yields `Match` objects one at a time

### When NOT to Use Regex
- Literal substring search: use `in`, `.find()`, `.index()`
- Fixed-string replacement: use `.replace()`
- Prefix/suffix checks: use `.startswith()`, `.endswith()`
- Simple splitting: use `.split()`

### Pitfalls to Avoid
1. **Catastrophic backtracking**: avoid nested quantifiers like `(a+)+`
2. **Over-engineering**: don't use regex when string methods suffice
3. **Forgetting raw strings**: always use `r"..."` for regex patterns